# Transient absorption case study of the chromophoric systems rc and rcg

## part 3 simultaneous target of rcg, rcgcr and gcrcg 

van Stokkum IHM, Wohlmuth C, Würthner F, Williams RM (2022) Energy transfer in supramolecular calix[4]arene—Perylene bisimide dye light harvesting building blocks: Resolving loss processes with simultaneous target analysis. Journal of Photochemistry and Photobiology 12:100154. doi:https://doi.org/10.1016/j.jpap.2022.100154

# Inspect experimental data


In [ ]:
from pyglotaran_extras import plot_data_overview

experiment_data = {
    "tas_rcg_dcm": "data/Krg_grga.ascii",
    "tas_gcrcg_dcm": "data/Krg_grgb.ascii",
    "guide_rcg_r1": "data/Krg_grgc.ascii",
    "guide_rcg_r2": "data/Krg_grgd.ascii",
    "guide_rcg_r3": "data/Krg_grge.ascii",
    "guide_rcg_r4": "data/Krg_grgf.ascii",
    "tas_rcgcr_dcm": "data/Krgr_grga.ascii",
    "guide_rcg_g": "data/Krg_grgg.ascii",
    "guide_gcrcg_g": "data/Krg_grgg.ascii",
    "guide_rcgcr_g": "data/Krg_grgg.ascii",
    "guide_gcrcg_r1": "data/Krg_grgc.ascii",
    "guide_gcrcg_r2": "data/Krg_grgd.ascii",
    "guide_gcrcg_r3": "data/Krg_grge.ascii",
    "guide_gcrcg_r4": "data/Krg_grgf.ascii",
    "guide_rcgcr_r1": "data/Krg_grgc.ascii",
    "guide_rcgcr_r2": "data/Krg_grgd.ascii",
    "guide_rcgcr_r3": "data/Krg_grge.ascii",
    "guide_rcgcr_r4": "data/Krg_grgf.ascii",
}

plot_data_overview(experiment_data["tas_gcrcg_dcm"], linlog=True, linthresh=1)

# Model and Parameter definition


In [ ]:
# from glotaran.utils.ipython import display_file

model_path = "models/model_target_rcg_gcrcg_rcgcr_refine.yml"
# display_file(model_path, syntax="yaml")

In [ ]:
parameters_file_path = "models/parameters_target_rcg_gcrcg_rcgcr_refine.yml"
# display_file(parameters_file_path, syntax="yaml")

# Optimization


- 3              4         8.9523e+03      1.40e-06       6.43e-03       7.56e-03 


In [ ]:
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme

scheme = Scheme(
    model_path,
    parameters_file_path,
    experiment_data,
    clp_link_tolerance=1.9,
    maximum_number_function_evaluations=7,
)
result = optimize(scheme)

## Inspect fit quality


In [ ]:
result

## Plot fitted traces


In [ ]:
from pyglotaran_extras.plotting.plot_traces import plot_fitted_traces
from pyglotaran_extras.plotting.plot_traces import select_plot_wavelengths

wavelengths = select_plot_wavelengths(
    result.data["tas_rcg_dcm"], equidistant_wavelengths=False, axes_shape=[6, 5]
)
fig3tr, axes = plot_fitted_traces(result, wavelengths, axes_shape=[6, 5], linlog=True, linthresh=1)

In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas

myFRLcolors = [
    "tab:grey",
    "tab:orange",
    "r",
    "tab:purple",
    "g",
    "k",
    "b",
    "m",
    "c",
    "y",
    "tab:brown",
]
custom_cycler = cycler(color=myFRLcolors)


def plot_concentration_and_spectra(result_dataset):
    # fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    plot_concentrations(result_dataset, axes[0], center_λ=0, linlog=True, cycler=custom_cycler)
    plot_sas(result_dataset, axes[1], cycler=custom_cycler)
    return fig, axes


fig, axes = plot_concentration_and_spectra(result.data["tas_rcg_dcm"])
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

# Plot result for interpretation

## Overview


In [ ]:
# from pyglotaran_extras import plot_overview

# plot_overview(result, linlog=True, linthresh=1, figure_only=False);

## Fit of the guidance SADS of gcrcg


In [ ]:
from pyglotaran_extras.plotting.style import PlotStyle

# fig, ax = plt.subplots(1, 1, figsize=(15, 7))
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.set_prop_cycle(PlotStyle().data_cycler_solid_dashed)
for data_set_suffix in ["r1", "r2", "r3", "r4", "g"]:
    dataset_name = f"guide_gcrcg_{data_set_suffix}"
    result.data[dataset_name].data.plot(label=f"{dataset_name} data", ax=ax)
    result.data[dataset_name].fitted_data.plot(label=f"{dataset_name} fitted data", ax=ax)

ax.legend()
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("SADS (a.u.)")
ax.set_title("Fit of the guidance SADS")
ax.axhline(0, color="k", linewidth=1)
# axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction",fontsize=16)

## Plot of the concentrations of rcg, rcgcr and gcrcg 

In [ ]:
# NB here one needs to select the species_concentration of a single wavelength
import xarray as xr
from cycler import cycler

# fig, ax = plt.subplots(1, 1, figsize=(15, 7))
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
myFRLcolors = ["tab:grey", "tab:orange", "r", "tab:purple", "g", "k"]
center_λ = 600


def plot_concentrations_compare(dataset=xr.Dataset, linestyle=str):
    plot_concentrations(
        dataset,
        ax=ax,
        center_λ=center_λ,
        cycler=cycler(color=myFRLcolors, linestyle=[linestyle] * 6),
        linlog=True,
        linthresh=1,
    )


plot_concentrations_compare(result.data["tas_rcg_dcm"], "-")
plot_concentrations_compare(result.data["tas_gcrcg_dcm"], "--")
plot_concentrations_compare(result.data["tas_rcgcr_dcm"], ":")


ax.set_xlabel("Time (ps)")
ax.set_ylabel("Population")
ax.legend(result.data["tas_rcg_dcm"].species.values);

## Coherent Artifact


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

irfas_plot_wavelength = 420

plot_coherent_artifact(
    result.data["tas_rcg_dcm"], time_range=(-0.3, 0.3), spectral=irfas_plot_wavelength
);

In [ ]:
plot_coherent_artifact(
    result.data["tas_gcrcg_dcm"], time_range=(-0.3, 0.3), spectral=irfas_plot_wavelength
);

In [ ]:
# NB clipped at the left, can we choose to plot the irf at another wavelength?
plot_coherent_artifact(
    result.data["tas_rcgcr_dcm"], time_range=(-0.3, 0.3), spectral=irfas_plot_wavelength
);

## Amplitude matrices of the target analysis for each dataset

In [ ]:
from pyglotaran_extras.inspect import show_a_matrixes

show_a_matrixes(result)